In [1]:
from __future__ import division, print_function, absolute_import

import tensorflow as tf


In [2]:

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
import ssl
data = input_data.read_data_sets('data/MNIST/', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/MNIST/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/MNIST/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting data/MNIST/t10k-images-idx3-ubyte.gz
Extracting data/MNIST/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
print("Size of:")
print("- Training-set:\t\t{}".format(len(data.train.labels)))
print("- Test-set:\t\t{}".format(len(data.test.labels)))
print("- Validation-set:\t{}".format(len(data.validation.labels)))

Size of:
- Training-set:		55000
- Test-set:		10000
- Validation-set:	5000


In [5]:
28*28

784

In [6]:
128*500

64000

In [7]:
# Training Parameters
learning_rate = 0.01
num_steps = 500
batch_size = 128
display_step = 10

# Network Parameters
num_input = 28*28 # MNIST data input (img shape: 28*28)
num_classes = 10 # MNIST total classes (0-9 digits)
dropout = 0.75 # Dropout, probability to keep units

# tf Graph input
X = tf.placeholder(tf.float32, [None, num_input])
Y = tf.placeholder(tf.float32, [None, num_classes])
keep_prob = tf.placeholder(tf.float32) # dropout (keep probability)

In [8]:
# Create some wrappers for simplicity
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)


def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')


In [9]:
# Create model
def conv_net(x, weights, biases, dropout):
    # MNIST data input is a 1-D vector of 784 features (28*28 pixels)
    # Reshape to match picture format [Height x Width x Channel]
    # Tensor input become 4-D: [Batch Size, Height, Width, Channel]
    x = tf.reshape(x, shape=[-1, 28, 28, 1])

    # Convolution Layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1, k=2)

    # Convolution Layer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [10]:
7*7*64

3136

In [11]:
# Store layers weight & bias
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, 10]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([10]))
}

# Construct model
logits = conv_net(X, weights, biases, keep_prob)
prediction = tf.nn.softmax(logits)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)


# Evaluate model
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [12]:
# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, num_steps+1):
        batch_x, batch_y = data.train.next_batch(batch_size)
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y, keep_prob: dropout})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y,
                                                                 keep_prob: 1.0})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    # Calculate accuracy for 256 MNIST test images
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: data.test.images[:256],
                                      Y: data.test.labels[:256],
                                      keep_prob: 1.0}))


Step 1, Minibatch Loss= 69305.4766, Training Accuracy= 0.195
Step 10, Minibatch Loss= 3529.1899, Training Accuracy= 0.773
Step 20, Minibatch Loss= 2590.7986, Training Accuracy= 0.773
Step 30, Minibatch Loss= 873.3868, Training Accuracy= 0.844
Step 40, Minibatch Loss= 514.4443, Training Accuracy= 0.906
Step 50, Minibatch Loss= 361.2151, Training Accuracy= 0.914
Step 60, Minibatch Loss= 296.4946, Training Accuracy= 0.867
Step 70, Minibatch Loss= 131.0615, Training Accuracy= 0.922
Step 80, Minibatch Loss= 294.8167, Training Accuracy= 0.906
Step 90, Minibatch Loss= 231.0658, Training Accuracy= 0.922
Step 100, Minibatch Loss= 75.5180, Training Accuracy= 0.938
Step 110, Minibatch Loss= 85.3986, Training Accuracy= 0.953
Step 120, Minibatch Loss= 102.5100, Training Accuracy= 0.953
Step 130, Minibatch Loss= 169.0390, Training Accuracy= 0.945
Step 140, Minibatch Loss= 307.9094, Training Accuracy= 0.914
Step 150, Minibatch Loss= 153.0907, Training Accuracy= 0.930
Step 160, Minibatch Loss= 44.3576

In [13]:
# Placeholder variable for the input images
x = tf.placeholder(tf.float32, shape=[None, 28*28], name='X')
# Reshape it into [num_images, img_height, img_width, num_channels]
x_image = tf.reshape(x, [-1, 28, 28, 1])

# Placeholder variable for the true labels associated with the images
y_true = tf.placeholder(tf.float32, shape=[None, 10], name='y_true')
y_true_cls = tf.argmax(y_true, dimension=1)

Instructions for updating:
Use the `axis` argument instead


In [14]:
def new_conv_layer(input, num_input_channels, filter_size, num_filters, name):
    
    with tf.variable_scope(name) as scope:
        # Shape of the filter-weights for the convolution
        shape = [filter_size, filter_size, num_input_channels, num_filters]

        # Create new weights (filters) with the given shape
        weights = tf.Variable(tf.truncated_normal(shape, stddev=0.05))

        # Create new biases, one for each filter
        biases = tf.Variable(tf.constant(0.05, shape=[num_filters]))

        # TensorFlow operation for convolution
        layer = tf.nn.conv2d(input=input, filter=weights, strides=[1, 1, 1, 1], padding='SAME')

        # Add the biases to the results of the convolution.
        layer += biases
        
        return layer, weights


In [15]:
def new_pool_layer(input, name):
    
    with tf.variable_scope(name) as scope:
        # TensorFlow operation for convolution
        layer = tf.nn.max_pool(value=input, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        
        return layer

In [16]:
def new_relu_layer(input, name):
    
    with tf.variable_scope(name) as scope:
        # TensorFlow operation for convolution
        layer = tf.nn.relu(input)
        
        return layer

In [17]:
def new_fc_layer(input, num_inputs, num_outputs, name):
    
    with tf.variable_scope(name) as scope:

        # Create new weights and biases.
        weights = tf.Variable(tf.truncated_normal([num_inputs, num_outputs], stddev=0.05))
        biases = tf.Variable(tf.constant(0.05, shape=[num_outputs]))
        
        # Multiply the input and weights, and then add the bias-values.
        layer = tf.matmul(input, weights) + biases
        
        return layer


In [18]:
# Convolutional Layer 1
layer_conv1, weights_conv1 = new_conv_layer(input=x_image, num_input_channels=1, filter_size=5, num_filters=6, 
                                            name ="conv1")

# Pooling Layer 1
layer_pool1 = new_pool_layer(layer_conv1, name="pool1")

# RelU layer 1
layer_relu1 = new_relu_layer(layer_pool1, name="relu1")

# Convolutional Layer 2
layer_conv2, weights_conv2 = new_conv_layer(input=layer_relu1, num_input_channels=6, filter_size=5, num_filters=16, 
                                            name= "conv2")

# Pooling Layer 2
layer_pool2 = new_pool_layer(layer_conv2, name="pool2")

# RelU layer 2
layer_relu2 = new_relu_layer(layer_pool2, name="relu2")

# Flatten Layer
num_features = layer_relu2.get_shape()[1:4].num_elements()
layer_flat = tf.reshape(layer_relu2, [-1, num_features])

# Fully-Connected Layer 1
layer_fc1 = new_fc_layer(layer_flat, num_inputs=num_features, num_outputs=128, name="fc1")

# RelU layer 3
layer_relu3 = new_relu_layer(layer_fc1, name="relu3")

# Fully-Connected Layer 2
layer_fc2 = new_fc_layer(input=layer_relu3, num_inputs=128, num_outputs=10, name="fc2")

In [19]:
layer_conv1

<tf.Tensor 'conv1/add:0' shape=(?, 28, 28, 6) dtype=float32>

In [20]:
# Use Softmax function to normalize the output
with tf.variable_scope("Softmax"):
    y_pred = tf.nn.softmax(layer_fc2)
    y_pred_cls = tf.argmax(y_pred, dimension=1)

In [21]:
# Use Cross entropy cost function
with tf.name_scope("cross_ent"):
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=layer_fc2, labels=y_true)
    cost = tf.reduce_mean(cross_entropy)

In [22]:
# Use Adam Optimizer
with tf.name_scope("optimizer"):
    optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)

In [23]:
# Accuracy
with tf.name_scope("accuracy"):
    correct_prediction = tf.equal(y_pred_cls, y_true_cls)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [24]:
# Initialize the FileWriter
writer = tf.summary.FileWriter("Training_FileWriter/")
writer1 = tf.summary.FileWriter("Validation_FileWriter/")

In [25]:
# Add the cost and accuracy to summary
tf.summary.scalar('loss', cost)
tf.summary.scalar('accuracy', accuracy)

# Merge all summaries together
merged_summary = tf.summary.merge_all()

In [29]:
num_epochs = 2
batch_size = 100
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import time

In [30]:
with tf.Session() as sess:
    # Initialize all variables
    sess.run(tf.global_variables_initializer())
    
    # Add the model graph to TensorBoard
    writer.add_graph(sess.graph)
    
    # Loop over number of epochs
    for epoch in range(num_epochs):
        
        start_time = time.time()
        train_accuracy = 0
        
        for batch in range(0, int(len(data.train.labels)/batch_size)):
            
            # Get a batch of images and labels
            x_batch, y_true_batch = data.train.next_batch(batch_size)
            
            # Put the batch into a dict with the proper names for placeholder variables
            feed_dict_train = {x: x_batch, y_true: y_true_batch}
            
            # Run the optimizer using this batch of training data.
            sess.run(optimizer, feed_dict=feed_dict_train)
            
            # Calculate the accuracy on the batch of training data
            train_accuracy += sess.run(accuracy, feed_dict=feed_dict_train)
            
            # Generate summary with the current batch of data and write to file
            summ = sess.run(merged_summary, feed_dict=feed_dict_train)
            writer.add_summary(summ, epoch*int(len(data.train.labels)/batch_size) + batch)
        
          
        train_accuracy /= int(len(data.train.labels)/batch_size)
        
        # Generate summary and validate the model on the entire validation set
        summ, vali_accuracy = sess.run([merged_summary, accuracy], feed_dict={x:data.validation.images, y_true:data.validation.labels})
        writer1.add_summary(summ, epoch)
        

        end_time = time.time()
        
        print("Epoch "+str(epoch+1)+" completed : Time usage "+str(int(end_time-start_time))+" seconds")
        print("\tAccuracy:")
        print ("\t- Training Accuracy:\t{}".format(train_accuracy))
        print ("\t- Validation Accuracy:\t{}".format(vali_accuracy))


Epoch 1 completed : Time usage 30 seconds
	Accuracy:
	- Training Accuracy:	0.7236909074302424
	- Validation Accuracy:	0.8866000175476074
Epoch 2 completed : Time usage 29 seconds
	Accuracy:
	- Training Accuracy:	0.9026727293838154
	- Validation Accuracy:	0.91839998960495


In [32]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)



Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [33]:
# Python optimisation variables
learning_rate = 0.01
epochs = 5
batch_size = 50

# declare the training data placeholders
# input x - for 28 x 28 pixels = 784 - this is the flattened image data that is drawn from 
# mnist.train.nextbatch()
x = tf.placeholder(tf.float32, [None, 784])
# dynamically reshape the input
x_shaped = tf.reshape(x, [-1, 28, 28, 1])
# now declare the output data placeholder - 10 digits
y = tf.placeholder(tf.float32, [None, 10])

In [41]:
x_shaped

<tf.Tensor 'Reshape_4:0' shape=(?, 28, 28, 1) dtype=float32>

In [65]:
def create_new_conv_layer(input_data, num_input_channels, num_filters, filter_shape, pool_shape, name):
    # setup the filter input shape for tf.nn.conv_2d
    conv_filt_shape = [filter_shape[0], filter_shape[1], num_input_channels,
                      num_filters]
    
    print ("conv filer:", len(conv_filt_shape))

    # initialise weights and bias for the filter
    weights = tf.Variable(tf.truncated_normal(conv_filt_shape, stddev=0.03),
                                      name=name+'_W')
    print("weights", weights.shape)
    bias = tf.Variable(tf.truncated_normal([num_filters]), name=name+'_b')

    print("bias", bias.shape)
    # setup the convolutional layer operation
    out_layer = tf.nn.conv2d(input_data, weights, [1, 1, 1, 1], padding='SAME')
    print ("output layer", out_layer.shape )

    # add the bias
    out_layer += bias
    print("output_layer", out_layer.shape)

    # apply a ReLU non-linear activation
    out_layer = tf.nn.relu(out_layer)
    print("output_layer after ReLU", out_layer.shape)


    # now perform max pooling
    ksize = [1, pool_shape[0], pool_shape[1], 1]
    strides = [1, 2, 2, 1]
    out_layer = tf.nn.max_pool(out_layer, ksize=ksize, strides=strides, 
                               padding='SAME')
    
    print("output_layer after Max pooling", out_layer.shape)
    print("-----------")

    return out_layer

In [66]:
layer1 = create_new_conv_layer(x_shaped, 1, 32, [5, 5], [2, 2], name='layer1')
layer2 = create_new_conv_layer(layer1, 32, 64, [5, 5], [2, 2], name='layer2')

conv filer: 4
weights (5, 5, 1, 32)
bias (32,)
output layer (?, 28, 28, 32)
output_layer (?, 28, 28, 32)
output_layer after ReLU (?, 28, 28, 32)
output_layer after Max pooling (?, 14, 14, 32)
-----------
conv filer: 4
weights (5, 5, 32, 64)
bias (64,)
output layer (?, 14, 14, 64)
output_layer (?, 14, 14, 64)
output_layer after ReLU (?, 14, 14, 64)
output_layer after Max pooling (?, 7, 7, 64)
-----------


In [67]:
layer1

<tf.Tensor 'MaxPool_22:0' shape=(?, 14, 14, 32) dtype=float32>

In [68]:
layer2

<tf.Tensor 'MaxPool_23:0' shape=(?, 7, 7, 64) dtype=float32>

In [69]:
flattened = tf.reshape(layer2, [-1, 7 * 7 * 64])

In [70]:
flattened

<tf.Tensor 'Reshape_6:0' shape=(?, 3136) dtype=float32>

In [71]:
# setup some weights and bias values for this layer, then activate with ReLU
wd1 = tf.Variable(tf.truncated_normal([7 * 7 * 64, 1000], stddev=0.03), name='wd1')
bd1 = tf.Variable(tf.truncated_normal([1000], stddev=0.01), name='bd1')
dense_layer1 = tf.matmul(flattened, wd1) + bd1
dense_layer1 = tf.nn.relu(dense_layer1)

In [72]:
wd1.shape

TensorShape([Dimension(3136), Dimension(1000)])

In [74]:
bd1.shape

TensorShape([Dimension(1000)])

In [75]:
dense_layer1.shape

TensorShape([Dimension(None), Dimension(1000)])

In [76]:
# another layer with softmax activations
wd2 = tf.Variable(tf.truncated_normal([1000, 10], stddev=0.03), name='wd2')
bd2 = tf.Variable(tf.truncated_normal([10], stddev=0.01), name='bd2')
dense_layer2 = tf.matmul(dense_layer1, wd2) + bd2
y_ = tf.nn.softmax(dense_layer2)

In [77]:
wd2.shape

TensorShape([Dimension(1000), Dimension(10)])

In [78]:
bd2.shape

TensorShape([Dimension(10)])

In [79]:
dense_layer2.shape

TensorShape([Dimension(None), Dimension(10)])

In [80]:
y_.shape

TensorShape([Dimension(None), Dimension(10)])

In [82]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=dense_layer2, labels=y))

In [85]:
cross_entropy

<tf.Tensor 'Mean_2:0' shape=() dtype=float32>

In [89]:
# add an optimiser
optimiser = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cross_entropy)
print(optimiser)

# define an accuracy assessment operation
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
print(correct_prediction)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(accuracy)

# setup the initialisation operator
init_op = tf.global_variables_initializer()



name: "Adam_4"
op: "NoOp"
input: "^Adam_4/update_layer1_W_10/ApplyAdam"
input: "^Adam_4/update_layer1_b_10/ApplyAdam"
input: "^Adam_4/update_layer2_W_10/ApplyAdam"
input: "^Adam_4/update_layer2_b_10/ApplyAdam"
input: "^Adam_4/update_wd1/ApplyAdam"
input: "^Adam_4/update_bd1/ApplyAdam"
input: "^Adam_4/update_wd2/ApplyAdam"
input: "^Adam_4/update_bd2/ApplyAdam"
input: "^Adam_4/Assign"
input: "^Adam_4/Assign_1"

Tensor("Equal_4:0", shape=(?,), dtype=bool)
Tensor("Mean_6:0", shape=(), dtype=float32)


In [92]:
with tf.Session() as sess:
    # initialise the variables
    sess.run(init_op)
    total_batch = int(len(mnist.train.labels) / batch_size)
    print("total_batch",total_batch)
    for epoch in range(epochs):
        print("epoch",epoch)
        avg_cost = 0
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size=batch_size)
            print("batch_x",batch_x.shape)
            print("batch_y", batch_y.shape)
            _, c = sess.run([optimiser, cross_entropy], 
                            feed_dict={x: batch_x, y: batch_y})
            
            print("c", c)
            avg_cost += c / total_batch
            print("avg_cost", avg_cost)
            print("---------------------")
        test_acc = sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels})
        print("Epoch:", (epoch + 1), "cost =", "{:.3f}".format(avg_cost), "test accuracy: {:.3f}".format(test_acc))
        print("====================================")

    print("\nTraining complete!")
    print(sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels}))

total_batch 1100
epoch 0
batch_x (50, 784)
batch_y (50, 10)
c 2.5357256
avg_cost 0.0023052050850608133
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 108.272
avg_cost 0.10073429887945001
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 21.486034
avg_cost 0.12026705741882324
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.628526
avg_cost 0.12265662648461081
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.3209367
avg_cost 0.12476656892082907
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.2986202
avg_cost 0.12685622366991908
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.2926693
avg_cost 0.12894046848470514
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.2959678
avg_cost 0.1310277119549838
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.3099139
avg_cost 0.13312763365832242
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.3119698
avg_cost 0.13522942434657703
-------------------

c 2.3068733
avg_cost 0.294017498926683
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.2946198
avg_cost 0.29610351692546505
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.3040545
avg_cost 0.2981981119242582
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.306288
avg_cost 0.30029473738236867
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.300729
avg_cost 0.30238630923357884
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.3257234
avg_cost 0.304500603242354
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.3108053
avg_cost 0.3066013353521174
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.2850013
avg_cost 0.3086786092411389
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.3041701
avg_cost 0.310773309360851
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.2924006
avg_cost 0.3128573099049656
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.308232
avg_cost 0.3149557026

c 2.3013043
avg_cost 0.4762413035739554
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.3157306
avg_cost 0.47834651318463434
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.2931767
avg_cost 0.48043121923099885
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.3087046
avg_cost 0.48253004160794366
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.2937503
avg_cost 0.4846152691407639
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.2872634
avg_cost 0.4866945994984022
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.30699
avg_cost 0.488791863051328
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.2920654
avg_cost 0.4908755588531496
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.2907724
avg_cost 0.49295807925137625
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.3025928
avg_cost 0.49505134539170714
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.3111734
avg_cost 0.49715

c 2.3081973
avg_cost 0.6582956088672988
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.313059
avg_cost 0.6603983898596333
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.298134
avg_cost 0.6624876026673753
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.2947161
avg_cost 0.6645737082307991
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.3022006
avg_cost 0.6666666178269822
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.2749186
avg_cost 0.6687347256053581
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.3002083
avg_cost 0.6708258240873166
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.3089597
avg_cost 0.6729248783805156
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.3031838
avg_cost 0.6750186818296262
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.3028195
avg_cost 0.677112154093656
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.3082676
avg_cost 0.6792105791

c 2.3111942
avg_cost 0.8402558504451412
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.3159869
avg_cost 0.8423612930557951
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.3358572
avg_cost 0.8444847995584668
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.3192747
avg_cost 0.846593231071126
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.3042612
avg_cost 0.8486880139871084
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.2992759
avg_cost 0.8507782647826462
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.3140602
avg_cost 0.8528819558837204
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.3050368
avg_cost 0.8549774438684644
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.3113976
avg_cost 0.8570787143707282
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.2881107
avg_cost 0.8591588150371212
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.293834
avg_cost 0.861244118

...

c 2.3140426
avg_cost 1.5240456351366924
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.3102238
avg_cost 1.5261458386074427
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.3092709
avg_cost 1.528245175751774
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.2767668
avg_cost 1.5303149637309001
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.2936378
avg_cost 1.5324000889604756
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.2978709
avg_cost 1.534489062482662
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.3229437
avg_cost 1.5366008294712428
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.3017416
avg_cost 1.5386933218349124
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.2999663
avg_cost 1.5407842003215457
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.2845418
avg_cost 1.5428610565445655
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.2963088
avg_cost 1.544948609

c 2.2854867
avg_cost 1.7059621906280533
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.300474
avg_cost 1.7080535305630093
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.3058443
avg_cost 1.7101497526602327
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.305716
avg_cost 1.7122458581490967
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.2921326
avg_cost 1.7143296150727723
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.3103738
avg_cost 1.716429954875601
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.2843838
avg_cost 1.7185066673972407
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.2950354
avg_cost 1.7205930631810986
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.3072531
avg_cost 1.7226905660195801
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.311069
avg_cost 1.7247915378483876
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.304678
avg_cost 1.726886699633

c 2.303582
avg_cost 1.8901624339277106
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.3216221
avg_cost 1.892272999503397
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.3174486
avg_cost 1.8943797709725132
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.2955883
avg_cost 1.8964666693860845
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.2982213
avg_cost 1.89855596152219
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.294944
avg_cost 1.9006422742930336
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.3098502
avg_cost 1.9027421381256806
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.3017685
avg_cost 1.9048346549814406
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.2912261
avg_cost 1.9069175878438092
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.3086352
avg_cost 1.9090163471482027
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.2971618
avg_cost 1.911104676073

c 2.3015833
avg_cost 2.0743898211825984
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.2936738
avg_cost 2.0764749791405426
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.2942677
avg_cost 2.078560677008196
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.305962
avg_cost 2.080657006177036
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.294889
avg_cost 2.082743268879978
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.30245
avg_cost 2.0848364051905555
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.289384
avg_cost 2.086917663270778
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.2963636
avg_cost 2.089005266536367
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.3016944
avg_cost 2.0910977159846924
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.296381
avg_cost 2.0931853350726053
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.3075998
avg_cost 2.0952831530570997
-

c 2.2754326
avg_cost 2.258455561074345
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.279954
avg_cost 2.2605282464894394
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.2938404
avg_cost 2.2626135559515532
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.29554
avg_cost 2.2647004105828037
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.299899
avg_cost 2.266791227947583
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.3029516
avg_cost 2.268884820287879
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.29777
avg_cost 2.2709737021272844
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.3115401
avg_cost 2.2730751022425575
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.3043091
avg_cost 2.2751699287241167
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.2762895
avg_cost 2.2772392827814287
---------------------
batch_x (50, 784)
batch_y (50, 10)
c 2.2963724
avg_cost 2.2793268940665516

In [ ]:
#http://adventuresinmachinelearning.com/convolutional-neural-networks-tutorial-tensorflow/